In [11]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [12]:
filename = "../../data/05/ABox.ttl"

In [13]:
text1 = '''CQ_5.1
What are the titles of the work `ALD-L1-work`? What are their types?
'''

query1 = '''
PREFIX ex: <http://purl.org/changes/object/data/05/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>  

SELECT ?title ?type ?content
WHERE {
    ?creation a frbroo:F28_Expression_Creation ;
              frbroo:R19_created_a_realisation_of ex:ALD-L1-work .
    ex:ALD-L1-work crm:P102_has_title ?title .
    ?title crm:P2_has_type ?type ;
                crm:P190_has_symbolic_content ?content .
}
'''

In [14]:
text2 = '''CQ_5.2
What are the subjects of the works that are not part of any parent work?
'''

query2 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?work ?subject
WHERE {
    ?creation a frbroo:F28_Expression_Creation ;
              frbroo:R19_created_a_realisation_of ?work ;
              frbroo:R17_created ?expression .
    ?work a frbroo:F1_Work .
    ?expression a frbroo:F2_Expression ;
                crm:P129_is_about ?subject .
    FILTER NOT EXISTS {
    ?parent frbroo:R10_has_member ?work .
  }
}
'''

In [15]:
text3 = '''CQ_5.3
What are the manifestations of the works which are members of parent works that are marine charts?
'''

query3 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?manifestation
WHERE {
    ?creation a frbroo:F28_Expression_Creation ;
              frbroo:R19_created_a_realisation_of ?work ;
              frbroo:R18_created ?manifestation .
    ?parent a frbroo:F15_Complex_Work ;
            frbroo:R10_has_member ?work ;
            crm:P2_has_type aat:300028309 .
}
'''

In [16]:
text4 = '''CQ_5.4
What are the parent works of the works that have either 'Europa' or 'tapiro' as their subject?
'''

query4 = '''
PREFIX ex: <http://purl.org/changes/object/data/05/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>  

SELECT ?parent ?work ?subject
WHERE {
  ?creation a frbroo:F28_Expression_Creation ;
              frbroo:R19_created_a_realisation_of ?work ;
              frbroo:R17_created ?expression .
    ?parent a frbroo:F15_Complex_Work ;
            frbroo:R10_has_member ?work .
    ?expression a frbroo:F2_Expression ;
                crm:P129_is_about ?subject .
FILTER (?subject IN (ex:sub-tapiro, ex:sub-europa))
}
'''

In [17]:
text5 = '''CQ_5.5
What are the manifestations that compose another manifestation?'''

query5 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 

SELECT ?manifestation1 ?manifestation2
WHERE {
  ?manifestation1 crm:P46_is_composed_of ?manifestation2 .
}
'''

In [18]:
text6 = '''CQ_5.6
What are the manifestations that depict other expressions?'''

query6 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 

SELECT ?manifestation ?expression
WHERE {
  ?manifestation crm:P62_depicts ?expression .
}
'''

In [19]:
text7 = '''CQ_5.7
What are the license statements referring to the manifestations?
'''

query7 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 

SELECT ?manifestation ?license_link
WHERE {
  ?license crm:P67_refers_to ?manifestation ;
    crm:P2_has_type aat:300435434 ;
    crm:P70i_is_documented_in ?license_link .
}
'''

In [20]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text4, query4),
           (text5, query5),
           (text6, query6),
           (text7, query7)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_5.1
What are the titles of the work `ALD-L1-work`? What are their types?

+-------------------------------------------------------------+-------------------------------------------+------------------------------------+
| title                                                       | type                                      | content                            |
|-------------------------------------------------------------+-------------------------------------------+------------------------------------|
| http://purl.org/changes/object/data/05/ALD-L1-work-title-01 | http://vocab.getty.edu/page/aat/300417204 | Amerigo Vespucci sveglia l'America |
| http://purl.org/changes/object/data/05/ALD-L1-work-title-02 | http://vocab.getty.edu/page/aat/300417207 | Amerigo Vespucci sveglia l'America |
+-------------------------------------------------------------+-------------------------------------------+------------------------------------+
CQ_5.2
What are the subjects of the works that are no